# Implementação de Transfer Learning com Arquitetura UNET

Nesse notebook, faremos o processo de transfer learning com uma Arquitetura U-Net, extraída do projeto "Automatic Delineation of Agricultural Fields from Sentinel-2 Images" da University of Twente

O projeto com o modelo que usamos encontra-se disponível nas referências abaixo.

## Referências:

- Resign M. et al, 2021: Automatic Delineation of Agricultural Fields from Sentinel-2 Images. Disponível em: https://github.com/resingm/field-boundary-delineation/blob/master/paper.pdf. Acesso em 9 de maio de 2024.

- Resign M. et al, 2021. Repositório GitHub, acesso em 9 de maio de 2024, https://github.com/resingm/field-boundary-delineation/tree/master

## Como executar

Para executar esse notebook com CPU ou GPU siga os passos abaixo:
1. Baixe este notebook e importe para o <a href="https://colab.google/">Google Colab</a> ou acesse o link: https://colab.research.google.com/drive/1N1qnfT22pUsegvDvoYWfJ_GFZevqfIg6?usp=sharing
2. No menu superior direito do colab, onde está escrito "Conectar" ou "Ligar", clique na seta ao lado e escolha a opção "Alterar o tipo de ambiente de execução"
3. Uma janela será aberta onde você pode escolher entre utilizar CPU ou GPU. Neste caso, apenas a execução utilizando opção A100 GPU vai funcionar.
4. Para rodar este notebook, alguns arquivos são necessários. Acesse https://drive.google.com/drive/folders/1vcymtzjDJqVRd3x0ivSywnouhkqotq5J?usp=sharing e baixe os quatro arquivos da pasta. Caso prefira, poderá salvá-los em seu Google Drive, criando uma cópia. Esses são os dados que serão utilizados no modelo ("X" e "Y" são os dados prontos e são suficientes caso deseje ir diretamente para o treinamento, pulando a etapa "Preparação final dos Dados". "imgs" e "masks" são os dados pré-processados, mas ainda não passados pela última etapa).

Caso tenha baixado localmente:
5. Aguarde o download dos arquivos. Após, no menu lateral esquerdo do colab clique no ícone de pasta (último ícone), e depois no primeiro ícone para upload de arquivos ("Fazer upload para o armazenamento da sessão").
6. Adicione os dois arquivos previamente baixados.

Pronto! Já está tudo preparado para execução.

<strong>OBS: Caso deseje alterar de CPU para GPU, ou vice-versa, durante a execução, a sessão será reiniciada e tudo deverá ser executado novamente.

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras
import pickle
import pandas as pd
import tensorflow as tf
from keras import Input
from keras.datasets import cifar10
from keras.utils import to_categorical, Sequence
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Conv2DTranspose, concatenate, Dropout, UpSampling2D
from sklearn.model_selection import train_test_split
import time


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Leitura dos dados

In [3]:
file_x = open('/content/drive/MyDrive/Satelite Data/loaded_df/X.pkl', 'rb')

X = pickle.load(file_x)

file_y = open('/content/drive/MyDrive/Satelite Data/loaded_df/Y.pkl', 'rb')

Y = pickle.load(file_y)

file_x.close()
file_y.close()


# Transfer Learning

O modelo deve ser rodado utlizando a A100 GPU

In [4]:
import numpy as np

batch_size = X.shape[0]

new_channel = np.ones((batch_size, 208, 208, 1))

new_X = np.concatenate((X, new_channel), axis=-1)


(2160, 208, 208, 4)


In [5]:
from keras import layers
from keras import Model
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop
from keras.layers import UpSampling2D, Conv2D

In [6]:
from keras.models import Model
from keras.layers import Input

### TRANSFER LEARNING

model_path = "/content/drive/MyDrive/Models/Unet3.h5"
model = load_model(model_path)

input_layer = Input(shape=(208, 208, 4))

outputs = {model.layers[0].name: input_layer}

for layer in model.layers[1:]:
    if 'concat' in layer.name:  
        concat_inputs = [outputs[l.name] for l in layer._inbound_nodes[0].inbound_layers]
        x = layer(concat_inputs)
    else:
        prev_layers = layer._inbound_nodes[0].inbound_layers
        if isinstance(prev_layers, list):
            layer_inputs = [outputs[l.name] for l in prev_layers]
            x = layer(layer_inputs[0] if len(layer_inputs) == 1 else layer_inputs)
        else:
            x = layer(outputs[prev_layers.name])

    outputs[layer.name] = x

new_model = Model(inputs=input_layer, outputs=x)

new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
new_model.summary()


In [ ]:
total_params = new_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in new_model.trainable_weights])

print(f"Total de parâmetros no modelo é de {total_params:,}.")
print(f"Total de parâmetros que podem ser treinados no modelo são {num_trainable_params:,}.")

Total de parâmetros no modelo é de 1,302,402.
Total de parâmetros que podem ser treinados no modelo são 1,298,434.


In [7]:
last_desired_layer = model.get_layer('batch_normalization_21')
last_output = last_desired_layer.get_output_at(1)
print("Output tensor dimension for node 0:", last_output.shape)


Output tensor dimension for node 0: (None, 104, 104, 96)


In [8]:
x = layers.Flatten()(last_output)
x = layers.Dropout(0.3)(last_output) 

x = layers.Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal', name="conv2d_19")(x)
x = layers.Conv2D(2, 1, padding='same', name="conv2d_20")(x)
x = UpSampling2D(size=(2, 2))(x)
x = layers.Activation('softmax', name='softmax_activation')(x)
cut_model = Model(new_model.input, x)
cut_model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
cut_model.summary()

In [ ]:
total_params = cut_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in cut_model.trainable_weights])

print(f"Total de parâmetros no modelo é de {total_params:,}.")
print(f"Total de parâmetros que podem ser treinados no modelo são {num_trainable_params:,}.")

Total de parâmetros no modelo é de 1,070,530.
Total de parâmetros que podem ser treinados no modelo são 1,067,138.


In [9]:
X_train, X_test, y_train, y_test = train_test_split(new_X, Y, test_size=0.3)

In [ ]:
results = cut_model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

In [ ]:
def plot_results(results):
  acc = results.history['accuracy']
  val_acc = results.history['val_accuracy']
  loss = results.history['loss']
  val_loss = results.history['val_loss']
  epochs = range(len(acc))
  plt.plot(epochs, acc, 'g', label='Precisão do conjunto de Treino')
  plt.plot(epochs, val_acc, 'b', label='Precisão do conjunto de Validação')
  plt.plot(epochs, loss, 'r', label='Perda do conjunto de Treino')
  plt.plot(epochs, val_loss, 'm', label='Perda do conjunto de Validação')
  plt.title('Precisão dos conjuntos de Treino e Validação')
  plt.xlabel('Épocas')
  plt.ylabel('Precisão/Perda')
  plt.legend(loc=0)
  plt.figure()
  plt.show()


plot_results(results)

In [13]:
def VisualizeResults(index):
    img = new_X[index]
    img = img[np.newaxis, ...]
    pred_y = cut_model.predict(img)
    pred_mask = tf.argmax(pred_y[0], axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    fig, arr = plt.subplots(1, 3, figsize=(15, 15))
    arr[0].imshow(new_X[index])
    arr[0].set_title('Imagem Original')
    arr[1].imshow(Y[index,:,:,0])
    arr[1].set_title('Máscara Verdadeira')
    arr[2].imshow(pred_mask[:,:,0])
    arr[2].set_title('Máscara Predita')

In [ ]:
index = np.random.randint(0, len(new_X))
VisualizeResults(index)